# 09 - 종합 실습

## (1) 환경 준비

### 1) 라이브러리 불러오기

In [1]:
# 판다스, 넘파이 라이브러리 불러오기
import pandas as pd
import numpy as np

### 2) 데이터 불러오기

다음 세개의 데이터를 불러와서 저장합시다.

* sales : https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv
* products : https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv
* customers : https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv

In [2]:
sales = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv')
products = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv')
customers = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv')

세 데이터프레임의 상위 5개 행을 조회하시오.

In [3]:
sales.head(5)

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID
0,107,2,2016-01-02,p1036481,2,2100,c150417
1,69,1,2016-01-02,p1152861,1,1091,c212716
2,69,7,2016-01-02,p1013161,1,2600,c212716
3,69,8,2016-01-02,p1005771,1,1650,c212716
4,69,11,2016-01-02,p1089531,1,2600,c212716


In [4]:
products.head(5)

,ProductID,ProductName,Category,SubCategory,CategoryOrd
0,p1052661,새우깡,간식,과자,3
1,p1054261,고구마스틱,간식,과자,3
2,p1097821,짱구,간식,과자,3
3,p1097831,감자칩,간식,과자,3
4,p1119071,뿌셔뿌셔,간식,과자,3


In [5]:
customers.head(5)

,CustomerID,RegisterDate,Gender,BirthYear
0,c328222,2014-09-25,F,1960
1,c281448,2013-06-18,F,1974
2,c038336,2003-10-10,F,1968
3,c084237,2007-03-09,F,1982
4,c162600,2010-06-14,F,1978


## (2) 데이터 집계, 수정, 조회하기

### 1) sales의 ProductID 별 판매량(Qty)과 판매액(Amt)의 합계를 tmp에 저장하고 조회하시오.

In [9]:
tmp = sales.groupby('ProductID')[['Qty','Amt']].sum()
tmp

,Qty,Amt
ProductID,,
p1001771,1055,3354827
p1002841,903,11011541
p1005621,906,2601703
p1005771,3963,7319963
p1005891,5194,10119037
...,...,...
p1246581,2180,2456346
p1255281,809,955633
p1256521,701,1241545


### 2)  1번 결과를 판매액 기준 내림차순으로 정렬하고 상위 5개 상품을 조회하시오.  
(매출액 상위 5개 상품을 조회하시오.)

In [11]:
tmp.sort_values(by='Amt', ascending=False).head(5)

,Qty,Amt
ProductID,,
p1072601,4058,18129067
p1178011,1653,14078818
p1002841,903,11011541
p1005891,5194,10119037
p1194801,990,7517664


### 3) customers의 성별(Gender) 별 고객 수를 조회하시오.

In [14]:
len(customers)

2243

In [18]:
customers['Gender'].value_counts()

F    2060
M     183
Name: Gender, dtype: int64

### 4) customers의 출생연도(BirthYear) 로 부터 나이(Age)를 계산해서 변수로 추가하시오.

In [25]:
year = pd.datetime.now().year
year

/var/folders/f4/fb3b_sqx3rvc3gfr73v6ck_00000gn/T/ipykernel_78920/3369913029.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  year = pd.datetime.now().year


2022

In [27]:
Age = year-customers['BirthYear']
Age

0       62
1       48
2       54
3       40
4       44
        ..
2238    41
2239    55
2240    45
2241    69
2242    41
Name: BirthYear, Length: 2243, dtype: int64

### 5) customers의 나이(Age)를 나이대(AgeGroup)로 변환하시오.

* [ , 30)  : 30미만
* [30, 40) : 30대
* [40, 50) : 40대
* [50, 60) : 50대
* [60, 70) : 60대
* [70, )   : 70이상

> [30, 40) 의 의미 : 30 <= ,  < 40

In [30]:
bin = [0,30,40,50,60,70,np.inf]
customers['AgeGroup'] = pd.cut(Age, bins=bin, labels = ['30미만','30대','40대','50대','60대','70이상'], 
                             right=False)
customers.head(5)

,CustomerID,RegisterDate,Gender,BirthYear,AgeGroup
0,c328222,2014-09-25,F,1960,60대
1,c281448,2013-06-18,F,1974,40대
2,c038336,2003-10-10,F,1968,50대
3,c084237,2007-03-09,F,1982,40대
4,c162600,2010-06-14,F,1978,40대


### 6) customers의 나이대(AgeGroup) 별 고객수를 구해 봅시다.

In [31]:
customers['AgeGroup'].value_counts()

40대     925
50대     749
60대     267
30대     186
70이상    110
30미만      6
Name: AgeGroup, dtype: int64

### 7) 매출액 top 5 상품명을 조회해 봅시다.

* 매출액은 sales에 있고, 상품명은 products에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.
    * 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
    * 2) 상품명 별 매출액 합계를 집계합니다. 
    * 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.


In [32]:
sales.head(5)

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID
0,107,2,2016-01-02,p1036481,2,2100,c150417
1,69,1,2016-01-02,p1152861,1,1091,c212716
2,69,7,2016-01-02,p1013161,1,2600,c212716
3,69,8,2016-01-02,p1005771,1,1650,c212716
4,69,11,2016-01-02,p1089531,1,2600,c212716


In [33]:
products.head(5)

,ProductID,ProductName,Category,SubCategory,CategoryOrd
0,p1052661,새우깡,간식,과자,3
1,p1054261,고구마스틱,간식,과자,3
2,p1097821,짱구,간식,과자,3
3,p1097831,감자칩,간식,과자,3
4,p1119071,뿌셔뿌셔,간식,과자,3


In [35]:
# 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
pop = pd.merge(sales, products, on='ProductID', how='inner')
pop.head(5)


,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,ProductName,Category,SubCategory,CategoryOrd
0,107,2,2016-01-02,p1036481,2,2100,c150417,순두부,반찬류,두부,1
1,137,4,2016-01-02,p1036481,2,2100,c280590,순두부,반찬류,두부,1
2,63,16,2016-01-03,p1036481,1,1050,c037915,순두부,반찬류,두부,1
3,135,3,2016-01-04,p1036481,3,3150,c100815,순두부,반찬류,두부,1
4,63,13,2016-01-06,p1036481,10,10500,c048405,순두부,반찬류,두부,1


In [42]:
# 2) 2) 상품명 별 매출액 합계를 집계합니다.
pop['take'] = pop['Qty'] * pop['Amt']
pop.groupby('ProductName')[['take']].sum()

,take
ProductName,
감귤컵푸딩,1651560
감자칩,5171700
고구마스틱,2969522
깻잎,5954897
낱개캔디,19203973
느타리버섯,5902105
당근,5684897
두부_대,17797123
두부_소,10166705


In [43]:
# 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.
pop.sort_values(by = 'take', ascending = False).head(5)


,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,ProductName,Category,SubCategory,CategoryOrd,take
54568,131,3,2016-09-26,p1006411,100,109100,c352349,통단팥빵,간식,빵,3,10910000
51853,3,1,2016-09-24,p1011291,200,51600,c043687,낱개캔디,간식,사탕류,3,10320000
54975,131,1,2016-09-26,p1006281,100,86400,c352349,소보루빵,간식,빵,3,8640000
54986,73,1,2016-10-11,p1006281,60,51840,c096617,소보루빵,간식,빵,3,3110400
53531,55,7,2016-09-01,p1012951,-41,-67650,c340631,애호박,채소,열매채소,5,2773650


### 8) 연령대 별 매출액을 조회합시다.

* 매출액은 sales에 있고, 연령대(AgeGroup)은 customers에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.


In [44]:
# 1) sales와 customers를  CustomerID 기준으로 inner 조인을 합시다.

pop2 = pd.merge(sales,customers, on='CustomerID', how='inner')
pop2.head(10)

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,RegisterDate,Gender,BirthYear,AgeGroup
0,107,2,2016-01-02,p1036481,2,2100,c150417,2010-03-03,F,1974,40대
1,107,1,2016-01-02,p1175481,1,1300,c150417,2010-03-03,F,1974,40대
2,185,1,2016-01-04,p1162631,1,4600,c150417,2010-03-03,F,1974,40대
3,67,2,2016-01-11,p1012751,1,1350,c150417,2010-03-03,F,1974,40대
4,201,3,2016-01-12,p1005891,1,1950,c150417,2010-03-03,F,1974,40대
5,201,5,2016-01-12,p1026741,2,6454,c150417,2010-03-03,F,1974,40대
6,31,1,2016-01-20,p1052661,1,1455,c150417,2010-03-03,F,1974,40대
7,123,1,2016-01-21,p1089531,1,2600,c150417,2010-03-03,F,1974,40대
8,71,5,2016-01-22,p1175481,1,1300,c150417,2010-03-03,F,1974,40대
9,71,10,2016-01-22,p1013161,1,2900,c150417,2010-03-03,F,1974,40대


In [45]:
# 2) AgeGroup 별 매출액 합계를 집계합니다.
pop2['take'] = pop2['Qty'] * pop2['Amt']
pop2


,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,RegisterDate,Gender,BirthYear,AgeGroup,take
0,107,2,2016-01-02,p1036481,2,2100,c150417,2010-03-03,F,1974,40대,4200
1,107,1,2016-01-02,p1175481,1,1300,c150417,2010-03-03,F,1974,40대,1300
2,185,1,2016-01-04,p1162631,1,4600,c150417,2010-03-03,F,1974,40대,4600
3,67,2,2016-01-11,p1012751,1,1350,c150417,2010-03-03,F,1974,40대,1350
4,201,3,2016-01-12,p1005891,1,1950,c150417,2010-03-03,F,1974,40대,1950
...,...,...,...,...,...,...,...,...,...,...,...,...
70007,193,5,2017-03-31,p1179241,2,4400,c017735,2002-07-31,F,1960,60대,8800
70008,193,6,2017-03-31,p1144661,1,2500,c017735,2002-07-31,F,1960,60대,2500
70009,193,7,2017-03-31,p1012581,1,1050,c017735,2002-07-31,F,1960,60대,1050
70010,193,9,2017-03-31,p1052661,1,1545,c017735,2002-07-31,F,1960,60대,1545


In [46]:
pop2.groupby('AgeGroup')[['take']].sum()

,take
AgeGroup,
30미만,198393
30대,19445744
40대,145359466
50대,129270847
60대,29372090
70이상,14902634


### 9) [심화]연령대 별, 상품 카테고리 매출 비중(%)을 조회합시다.

* 1) 세 데이터프레임을 merge 해야 합니다.
    * merge는 한번에 두개씩만 가능하므로, 두번 수행합니다.
* 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다. 
    * 하나의 연령대 값에 대해서, 
        * 상품 카테고리(Category)별 매출액 합계를 구합니다.
        * 연령대에 해당하는 전체 매출액으로 나눠서, 카테고리별 비율을 계산합니다.
        * 결과를 출력합니다.



In [49]:
# 1) 세 데이터프레임을 merge 
pop3 = pd.merge(pop, customers, on='CustomerID', how='inner')
pop3.head(5)


,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,ProductName,Category,SubCategory,CategoryOrd,take,RegisterDate,Gender,BirthYear,AgeGroup
0,107,2,2016-01-02,p1036481,2,2100,c150417,순두부,반찬류,두부,1,4200,2010-03-03,F,1974,40대
1,197,5,2017-01-24,p1036481,1,1050,c150417,순두부,반찬류,두부,1,1050,2010-03-03,F,1974,40대
2,251,4,2016-07-15,p1152861,2,2182,c150417,포토아이스크림,유제품,아이스크림,4,4364,2010-03-03,F,1974,40대
3,71,10,2016-01-22,p1013161,1,2900,c150417,느타리버섯,채소,버섯,5,2900,2010-03-03,F,1974,40대
4,69,7,2016-09-13,p1013161,1,2950,c150417,느타리버섯,채소,버섯,5,2950,2010-03-03,F,1974,40대


In [72]:
# 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다
tot = pop3.groupby('Category')[['take']].sum()



In [66]:
a

40대     31890
50대     23943
60대      7045
30대      4471
70이상     2586
30미만       77
Name: AgeGroup, dtype: int64

In [70]:
pop3[pop3['AgeGroup']=='40대'].groupby('Category')[['take']].sum()

,take
Category,
간식,36336616
과일,27023317
반찬류,17220248
유제품,36885538
채소,27893747


In [78]:
a = pop3['AgeGroup'].value_counts()
for i in a.index : 
    print(i, "\n" , pop3[pop3['AgeGroup'] == i].groupby('Category')[['take']].sum()/tot, "\n")
    

40대 
               take
Category          
간식        0.532193
과일        0.431656
반찬류       0.340092
유제품       0.467137
채소        0.357276 

50대 
               take
Category          
간식        0.376117
과일        0.379343
반찬류       0.380784
유제품       0.371883
채소        0.399592 

60대 
               take
Category          
간식        0.018264
과일        0.086423
반찬류       0.161303
유제품       0.075894
채소        0.109571 

30대 
               take
Category          
간식        0.034631
과일        0.053850
반찬류       0.040453
유제품       0.065536
채소        0.083087 

70이상 
               take
Category          
간식        0.038539
과일        0.048490
반찬류       0.076058
유제품       0.019159
채소        0.049591 

30미만 
               take
Category          
간식        0.000256
과일        0.000238
반찬류       0.001311
유제품       0.000391
채소        0.000882 



In [82]:
pop3[pop3['AgeGroup']=='40대']['take'].sum()

145359466

In [87]:
# 안이쁜듯 .. 코드가..
a = pop3['AgeGroup'].value_counts()
for i in a.index : 
    print(i, "\n" , pop3[pop3['AgeGroup'] == i].groupby('Category')[['take']].sum()
          / pop3[pop3['AgeGroup']==i]['take'].sum(), "\n")
    

40대 
               take
Category          
간식        0.249978
과일        0.185907
반찬류       0.118467
유제품       0.253754
채소        0.191895 

50대 
               take
Category          
간식        0.198654
과일        0.183710
반찬류       0.149149
유제품       0.227153
채소        0.241334 

60대 
               take
Category          
간식        0.042456
과일        0.184204
반찬류       0.278067
유제품       0.204024
채소        0.291249 

30대 
               take
Category          
간식        0.121596
과일        0.173365
반찬류       0.105333
유제품       0.266115
채소        0.333590 

70이상 
               take
Category          
간식        0.176568
과일        0.203699
반찬류       0.258418
유제품       0.101512
채소        0.259803 

30미만 
               take
Category          
간식        0.088007
과일        0.075103
반찬류       0.334533
유제품       0.155429
채소        0.346928 

